# Algorithms for direct rending of quasicrystal tilings

[Quasicrystals](https://en.wikipedia.org/wiki/Quasicrystal) are aperiodic but regular tilings of space, and 2D quasicrystals have been used for centuries in [Islamic art](https://en.wikipedia.org/wiki/Girih_tiles). I find these patterns beautiful, and I think more people shuld know how to render them. 

You can render 2D quasicrystals [as a sum of plane waves](http://mainisusuallyafunction.blogspot.com/2011/10/quasicrystals-as-sums-of-waves-in-plane.html). They can also be constructed using aperiodic tilings (e.g. 4-fold [Ammann tiles](https://en.wikipedia.org/wiki/Robert_Ammann) and 5-fold [Penrose Tiles](https://en.wikipedia.org/wiki/Penrose_tiling)), and there are fractal constructions that recursively subdivide a set of tiles using a smaller set of the same tiles (e.g. [5-](https://mathworld.wolfram.com/PenroseTiles.html) and [7-](https://link.springer.com/article/10.1007/s11224-018-1083-7?shared-article-renderer)fold tilings). Many more constructions can be found on the [Tilings Encyclopedia](https://tilings.math.uni-bielefeld.de/). 

This post focuses on the ["cut and project"](https://en.wikipedia.org/wiki/Quasicrystal#Mathematics) construction, which is easy to implement in ocde. 

## The cut-and-project construction

The simplest way to render crisp, geometric quasicrystals is the ["cut and project"](https://en.wikipedia.org/wiki/Quasicrystal#Mathematics) construction. This views the quasicrystal as a two-dimensional projection of a planar slice through a N-dimensional hyperlattice. Let's implement this algorithm. 

### (Hyper) pixels in a (hyper) lattice

First, we need to define the idea of a hyperlattice. This is a N-dimensional generalization of a cubic crystal lattice. It tiles ND space using ND hypercubes. We can represent the center of each hypercube (hypercell? hyperpixel?) as a N-D vector of integers

$$
\mathbf q_0 = [ x_1, ... x_N ],\,\,x_i\in\mathbb Z
$$

The associated N-cube, then, includes all points on a unit hypercube $\left[-\tfrac 1 2, \tfrac 1 2\right]^N$ offset to location $\mathbf q_0$:

$$
\mathbf q = \left[-\tfrac 1 2, \tfrac 1 2\right]^N + \mathbf q_0
$$

### An irrational projection onto a 2D plane

To construct the quasicrystal, we cut through this ND lattice with a 2D plane. To get a aperiodic slice, this plane must not align with any periodic direction in the hyperlattice. A good choice is to project each direction of the hyperlattice onto an evenly-spaced set of basis directions, centred at the origin. We define a 2D $(s,t)$ plane $\mathcal P$ in terms of two N-D basis vectors, $\mathbf u, \mathbf v \in \mathbb R^N$. 

$$
\begin{aligned}
\mathbf p &= \mathbf u \cdot s + \mathbf v \cdot t,\,\,s,t\in\mathbb R
\\
\mathbf u &= [ \cos(\theta_0), \dots , \cos(\theta_{N-1}) ]
\\
\mathbf v &= [ \sin(\theta_0), \dots , \sin(\theta_{N-1}) ]
\\
\theta_i &= \frac {\pi\cdot i}{N}
\end{aligned}
$$

To render the quasicrystal, cut the N-D lattice along this plane, and project the exposed surface of this cut [isometrically](https://en.wikipedia.org/wiki/Isometric_projection) onto the 2D (s,t) plane \mathcal P. I've searched for an elegant algorithm for this to no avail. It seems like there should be some generalization of e.g. [Bresenham's line algorithm](https://en.wikipedia.org/wiki/Bresenham%27s_line_algorithm) to finding the hypercubes (hyperpixels?) associated with the cut. But, inelegant solutions work: it suffices to find the set of N-cubes in the ND lattice that intersect this plane. 

$$
\mathcal Q = \left\{ \mathbf q_0 \bigm| \mathcal P \cap \mathbf q\ne \varnothing \right\}
$$

### Algorithm 1: brute-force

An easy to code (but terible) way to find points in \mathcal Q is by a brute-force search: Check lots of points in the s,t plane, and quantize them to the nearest vertex \mathbf q_0 of the hyperlattice: 

$$
\mathcal Q = \left\{ \mathbf q_0 \bigm| \exists (s,t)\in\mathbb R^2 \text{ s.t. } \left \lfloor \mathbf u s + \mathbf v t \right \rceil =  \mathbf q_0 \right\},
$$

where \lfloor\cdot\rceil denotes rounding to the nearest integer vector. Here is some Python code that will get the job done: 

<!-- HTML generated using hilite.me --><div style="background: #ffffff; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><table><tr><td><pre style="margin: 0; line-height: 125%"> 1
 2
 3
 4
 5
 6
 7
 8
 9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29</pre></td><td><pre style="margin: 0; line-height: 125%"><span style="color: #008800; font-weight: bold">from</span> <span style="color: #0e84b5; font-weight: bold">scipy.spatial.distance</span> <span style="color: #008800; font-weight: bold">import</span> squareform, pdist
<span style="color: #008800; font-weight: bold">from</span> <span style="color: #0e84b5; font-weight: bold">pylab</span> <span style="color: #008800; font-weight: bold">import</span> <span style="color: #333333">*</span>
<span style="color: #008800; font-weight: bold">from</span> <span style="color: #0e84b5; font-weight: bold">numpy</span> <span style="color: #008800; font-weight: bold">import</span> <span style="color: #333333">*</span> 

N <span style="color: #333333">=</span> <span style="color: #0000DD; font-weight: bold">4</span>                      <span style="color: #888888"># Dimension of crystal</span>
θ <span style="color: #333333">=</span> linspace(<span style="color: #0000DD; font-weight: bold">0</span>,pi,N<span style="color: #333333">+</span><span style="color: #0000DD; font-weight: bold">1</span>)[<span style="color: #0000DD; font-weight: bold">1</span>:] <span style="color: #888888"># Arrange basis vectors evenly on [0,π)</span>
A <span style="color: #333333">=</span> array([cos(θ),sin(θ)]) <span style="color: #888888"># Convert from complex notation</span>

L  <span style="color: #333333">=</span> <span style="color: #0000DD; font-weight: bold">501</span>  <span style="color: #888888"># Number of grid points to search over</span>
dx <span style="color: #333333">=</span> <span style="color: #0000DD; font-weight: bold">5</span>    <span style="color: #888888"># Size of square portion of plane to check</span>

<span style="color: #888888"># Generate the search grid</span>
x  <span style="color: #333333">=</span> linspace(<span style="color: #333333">-</span>dx,dx,L)
xy <span style="color: #333333">=</span> array(meshgrid(x,x))<span style="color: #333333">.</span>reshape(<span style="color: #0000DD; font-weight: bold">2</span>,L<span style="color: #333333">*</span>L)

<span style="color: #888888"># Collapse all grid points to the nearest hyper-cell </span>
p  <span style="color: #333333">=</span> unique(np<span style="color: #333333">.</span>round(A<span style="color: #333333">.</span>T<span style="color: #555555; font-weight: bold">@xy</span>),axis<span style="color: #333333">=</span><span style="color: #0000DD; font-weight: bold">1</span>)

<span style="color: #008800; font-weight: bold">def</span> <span style="color: #0066BB; font-weight: bold">plotpoints</span>(p):
    u  <span style="color: #333333">=</span> pinv(A)<span style="color: #333333">.</span>T<span style="color: #555555; font-weight: bold">@p</span>
    D  <span style="color: #333333">=</span> squareform(pdist(p<span style="color: #333333">.</span>T))
    e  <span style="color: #333333">=</span> {<span style="color: #007020">tuple</span>(<span style="color: #007020">sorted</span>(e)) <span style="color: #008800; font-weight: bold">for</span> e <span style="color: #000000; font-weight: bold">in</span> <span style="color: #007020">zip</span>(<span style="color: #333333">*</span>where(<span style="color: #007020">abs</span>(D<span style="color: #333333">-</span><span style="color: #0000DD; font-weight: bold">1</span>)<span style="color: #333333">&lt;</span><span style="color: #0000DD; font-weight: bold">1</span>e<span style="color: #333333">-</span><span style="color: #0000DD; font-weight: bold">6</span>))}
    xy <span style="color: #333333">=</span> concatenate([[u[:,a],u[:,b],[NaN,NaN]] <span style="color: #008800; font-weight: bold">for</span> a,b <span style="color: #000000; font-weight: bold">in</span> e])
    plot(<span style="color: #333333">*</span>xy<span style="color: #333333">.</span>T,lw<span style="color: #333333">=</span><span style="color: #6600EE; font-weight: bold">0.5</span>,color<span style="color: #333333">=</span><span style="background-color: #fff0f0">&#39;k&#39;</span>)
    axis(<span style="background-color: #fff0f0">&quot;equal&quot;</span>)
    axis(<span style="background-color: #fff0f0">&quot;off&quot;</span>)
    
plotpoints(p)
savefig(<span style="background-color: #fff0f0">&quot;qq1_n%d.png&quot;</span><span style="color: #333333">%</span>N)
</pre></td></tr></table></div>



This approach is massively inefficient: many $(s,t)$ points map to the same lattice point $\mathbf q$, and it will miss points if the grid resolution is too coarse. But, for small renderings and run-once code, it does the job.

### Algorithm 2: testing for intersections

A better way is to start with a point that you know touches the plane. Then, "grow" the crystal from this point by checking if adjacent sites on the lattice also intersect the cutting plane. The plane defined above always intersects the N-cube at \mathbf q=[0,..,0], so this is a good place to start. [This mathematics stack exchange question](https://math.stackexchange.com/questions/3839170/check-whether-a-2d-plane-intersects-a-hypercube/3842277#3842277) gives two ways to check for cube-plane intersecton. They include *(a)* using linear programming, and *(b)* checking directly based on a geometric solution. Both are slower than I'd like, and I suspect there is an even faster/more elegant solution out there. 


#### Algorithm 2a: test with linear programming

As per [this answer](https://math.stackexchange.com/a/3839228/223565), we can check for plane-cube intersection using Linear Programming (LP). Mirko Stojiljković's ["*Hands-On Linear Programming: Optimization With Python*"](https://realpython.com/linear-programming-python/) is a good, accessible introduction to linear programming in Python. The code below searches for N-cubes intersecting the cutting plane iteratively. It includes optimizations, like memoizing the intersection tests, and exploting the N-fold symmetry to recycle previous test results. 

<!-- HTML generated using hilite.me --><div style="background: #ffffff; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><table><tr><td><pre style="margin: 0; line-height: 125%"> 1
 2
 3
 4
 5
 6
 7
 8
 9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47</pre></td><td><pre style="margin: 0; line-height: 125%"><span style="color: #008800; font-weight: bold">from</span> <span style="color: #0e84b5; font-weight: bold">scipy.optimize</span> <span style="color: #008800; font-weight: bold">import</span> linprog

<span style="color: #888888"># Number of search iterations</span>
Ndepth <span style="color: #333333">=</span> <span style="color: #0000DD; font-weight: bold">10</span> 

<span style="color: #888888"># Directions in which to search</span>
D <span style="color: #333333">=</span> int32(concatenate([eye(N),<span style="color: #333333">-</span>eye(N)])) 

<span style="color: #888888"># Matrix to rotate a lattice point π/N in 2D plane</span>
S <span style="color: #333333">=</span> zeros((N,N))
S[ <span style="color: #333333">-</span><span style="color: #0000DD; font-weight: bold">1</span>,<span style="color: #0000DD; font-weight: bold">0</span> ] <span style="color: #333333">=</span> <span style="color: #333333">-</span><span style="color: #0000DD; font-weight: bold">1</span>
S[:<span style="color: #333333">-</span><span style="color: #0000DD; font-weight: bold">1</span>,<span style="color: #0000DD; font-weight: bold">1</span>:] <span style="color: #333333">=</span> eye(N<span style="color: #333333">-</span><span style="color: #0000DD; font-weight: bold">1</span>)

<span style="color: #888888"># Matrix encoding LHS of linear inequalities for LP</span>
Aub <span style="color: #333333">=</span> concatenate([ A<span style="color: #333333">.</span>T,<span style="color: #333333">-</span>A<span style="color: #333333">.</span>T])

<span style="color: #888888"># Use linear programming to see if plane</span>
<span style="color: #888888"># intersects hypercube ±0.5 of p</span>
<span style="color: #008800; font-weight: bold">def</span> <span style="color: #0066BB; font-weight: bold">check_lp</span>(q):
    <span style="color: #008800; font-weight: bold">return</span> linprog(c<span style="color: #333333">=</span>[<span style="color: #0000DD; font-weight: bold">1</span>,<span style="color: #0000DD; font-weight: bold">1</span>],A_ub<span style="color: #333333">=</span>Aub,
        b_ub<span style="color: #333333">=</span>concatenate([<span style="color: #333333">.</span><span style="color: #0000DD; font-weight: bold">5</span><span style="color: #333333">+</span>q,<span style="color: #333333">.</span><span style="color: #0000DD; font-weight: bold">5</span><span style="color: #333333">-</span>q]),
        method<span style="color: #333333">=</span><span style="background-color: #fff0f0">&quot;revised simplex&quot;</span>,
        options<span style="color: #333333">=</span>{<span style="background-color: #fff0f0">&quot;tol&quot;</span>:<span style="color: #0000DD; font-weight: bold">1</span>e<span style="color: #333333">-</span><span style="color: #0000DD; font-weight: bold">9</span>})<span style="color: #333333">.</span>success

<span style="color: #888888"># Cache results to save time (memoization)</span>
cache <span style="color: #333333">=</span> {}
<span style="color: #008800; font-weight: bold">def</span> <span style="color: #0066BB; font-weight: bold">checkcrystal_lp</span>(q):
    <span style="color: #888888"># Convert test point to immutable tuple for cache key</span>
    k <span style="color: #333333">=</span> <span style="color: #007020">tuple</span>(int32(q)) 
    <span style="color: #008800; font-weight: bold">if</span> <span style="color: #000000; font-weight: bold">not</span> k <span style="color: #000000; font-weight: bold">in</span> cache:
        <span style="color: #888888"># Recompute intersection test if not in cache</span>
        <span style="color: #888888"># Use symmetry: reduce all tests to points in 1st sector</span>
        h <span style="color: #333333">=</span> angle(A<span style="color: #555555; font-weight: bold">@q</span><span style="color: #FF0000; background-color: #FFAAAA">@</span>[<span style="color: #0000DD; font-weight: bold">1</span>,<span style="color: #0000DD; font-weight: bold">1</span>j])
        cache[k]<span style="color: #333333">=</span>check_lp(q) <span style="color: #008800; font-weight: bold">if</span> <span style="color: #0000DD; font-weight: bold">0</span><span style="color: #333333">&lt;=</span>h<span style="color: #333333">&lt;=</span>(pi<span style="color: #333333">/</span>N<span style="color: #333333">*</span><span style="color: #6600EE; font-weight: bold">1.1</span>) <span style="color: #008800; font-weight: bold">else</span> checkcrystal_lp(S<span style="color: #555555; font-weight: bold">@q</span>)
    <span style="color: #008800; font-weight: bold">return</span> cache[k]

<span style="color: #888888"># Start with a seed point at zero and build outwards</span>
Q,allQ <span style="color: #333333">=</span> {(<span style="color: #0000DD; font-weight: bold">0</span>,)<span style="color: #333333">*</span>N},<span style="color: #007020">set</span>() 

<span style="color: #888888"># Iteratively search in each direction to add points</span>
<span style="color: #008800; font-weight: bold">for</span> i <span style="color: #000000; font-weight: bold">in</span> <span style="color: #007020">range</span>(Ndepth):
    Q <span style="color: #333333">=</span> {<span style="color: #007020">tuple</span>(q) <span style="color: #008800; font-weight: bold">for</span> p <span style="color: #000000; font-weight: bold">in</span> Q <span style="color: #008800; font-weight: bold">for</span> q <span style="color: #000000; font-weight: bold">in</span> D<span style="color: #333333">+</span>p <span style="color: #008800; font-weight: bold">if</span> checkcrystal_lp(q) }
    allQ <span style="color: #333333">|=</span> Q

figure(figsize<span style="color: #333333">=</span>(<span style="color: #0000DD; font-weight: bold">10</span>,<span style="color: #0000DD; font-weight: bold">10</span>))
plotpoints(array(<span style="color: #007020">list</span>(allQ))<span style="color: #333333">.</span>T)
savefig(<span style="background-color: #fff0f0">&quot;qq2_n%d.png&quot;</span><span style="color: #333333">%</span>N)
</pre></td></tr></table></div>


#### Algorithm 2b: testing using a geometric solution

[This answer](https://math.stackexchange.com/a/3842277/223565) outlines another way to test for plane-cube intersection. It involves projecting the problem onto a space where the 2D plane collapses to a point at the origin, and checking whether the shadow of the N-cube in this subspace contains this origin point. There are a few ways to do this. For example, you could calculate the convex hull of the N-2 projection, and test if it contains the origin. The code below uses another approach: It checks whether any of the N-2 sub-facets of the projected N-cube contains the origin, stopping early if it finds one. 





<!-- HTML generated using hilite.me --><div style="background: #ffffff; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><table><tr><td><pre style="margin: 0; line-height: 125%"> 1
 2
 3
 4
 5
 6
 7
 8
 9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68</pre></td><td><pre style="margin: 0; line-height: 125%"><span style="color: #008800; font-weight: bold">from</span> <span style="color: #0e84b5; font-weight: bold">scipy.linalg</span> <span style="color: #008800; font-weight: bold">import</span> null_space
<span style="color: #008800; font-weight: bold">from</span> <span style="color: #0e84b5; font-weight: bold">itertools</span> <span style="color: #008800; font-weight: bold">import</span> product, combinations

<span style="color: #888888"># Null space: collapses cutting plane to point at origin</span>
C <span style="color: #333333">=</span> null_space(A)<span style="color: #333333">.</span>T

<span style="color: #888888"># All vertices of an N-cube</span>
O <span style="color: #333333">=</span> array([<span style="color: #333333">*</span>product(<span style="color: #333333">*</span>([<span style="color: #0000DD; font-weight: bold">0</span>,<span style="color: #0000DD; font-weight: bold">1</span>],)<span style="color: #333333">*</span>N)])<span style="color: #333333">-</span><span style="color: #6600EE; font-weight: bold">0.5</span>

<span style="color: #888888"># All vertices of a 2-cube (used to generate N-2 sub-facets)</span>
F <span style="color: #333333">=</span> array([<span style="color: #333333">*</span>product(<span style="color: #333333">*</span>([<span style="color: #0000DD; font-weight: bold">0</span>,<span style="color: #0000DD; font-weight: bold">1</span>],)<span style="color: #333333">*</span><span style="color: #0000DD; font-weight: bold">2</span>)])

<span style="color: #888888"># A single corner (&quot;origin&quot; vertex) of an N-2 cube</span>
<span style="color: #888888"># plus N-2 vectors defining the other points on the N-2 cube</span>
<span style="color: #888888"># Used to test if sub-facet contains the origins</span>
i <span style="color: #333333">=</span> [<span style="color: #0000DD; font-weight: bold">0</span>]<span style="color: #333333">+</span>[<span style="color: #333333">*</span><span style="color: #0000DD; font-weight: bold">2</span><span style="color: #333333">**</span>arange(N<span style="color: #333333">-</span><span style="color: #0000DD; font-weight: bold">2</span>)]
G <span style="color: #333333">=</span> array([<span style="color: #333333">*</span>product(<span style="color: #333333">*</span>([<span style="color: #0000DD; font-weight: bold">0</span>,<span style="color: #0000DD; font-weight: bold">1</span>],)<span style="color: #333333">*</span>(N<span style="color: #333333">-</span><span style="color: #0000DD; font-weight: bold">2</span>))])[i,:]

<span style="color: #888888"># All possible pairs of dimension for generating</span>
<span style="color: #888888"># all possible N-2 sub-facets of the N-cube</span>
I <span style="color: #333333">=</span> [<span style="color: #333333">*</span>combinations(<span style="color: #007020">range</span>(N),<span style="color: #0000DD; font-weight: bold">2</span>)]

<span style="color: #888888"># Given a list of N-cube vertices, construct a list</span>
<span style="color: #888888"># of all N-2-cube sub-facets.</span>
subfacets <span style="color: #333333">=</span> []
<span style="color: #008800; font-weight: bold">for</span> ij <span style="color: #000000; font-weight: bold">in</span> I:
    <span style="color: #888888"># For every pair of directions, and for every 2-cube</span>
    <span style="color: #888888"># in each pair of directions... Generate all possible</span>
    <span style="color: #888888"># N-2-cube sub-facets.</span>
    v0 <span style="color: #333333">=</span> F<span style="color: #555555; font-weight: bold">@eye</span>(N)[ij,:]
    v1 <span style="color: #333333">=</span> G<span style="color: #555555; font-weight: bold">@eye</span>(N)[[<span style="color: #333333">*</span>{<span style="color: #333333">*</span><span style="color: #007020">range</span>(N)}<span style="color: #333333">-</span>{<span style="color: #333333">*</span>ij}]]
    subfacets<span style="color: #333333">.</span>extend(v0[:,<span style="color: #008800; font-weight: bold">None</span>,:]<span style="color: #333333">+</span>v1[<span style="color: #008800; font-weight: bold">None</span>,:,:])
vxid <span style="color: #333333">=</span> int32(subfacets)<span style="color: #555555; font-weight: bold">@2</span><span style="color: #333333">**</span>arange(N,dtype<span style="color: #333333">=</span><span style="background-color: #fff0f0">&#39;i&#39;</span>)

<span style="color: #888888"># Chec if a sub-facet contains the origin</span>
<span style="color: #008800; font-weight: bold">def</span> <span style="color: #0066BB; font-weight: bold">subfacet_contains</span>(v):
    p0 <span style="color: #333333">=</span> v[<span style="color: #0000DD; font-weight: bold">0</span>]
    vk <span style="color: #333333">=</span> v[<span style="color: #0000DD; font-weight: bold">1</span>:,:]<span style="color: #333333">-</span>p0
    q  <span style="color: #333333">=</span> <span style="color: #333333">-</span>np<span style="color: #333333">.</span>linalg<span style="color: #333333">.</span>solve(vk<span style="color: #555555; font-weight: bold">@vk</span><span style="color: #333333">.</span>T,vk<span style="color: #555555; font-weight: bold">@p0</span>)
    <span style="color: #008800; font-weight: bold">return</span> <span style="color: #007020">all</span>(q<span style="color: #333333">&gt;=</span><span style="color: #0000DD; font-weight: bold">0</span>) <span style="color: #000000; font-weight: bold">and</span> <span style="color: #007020">all</span>(q<span style="color: #333333">&lt;=</span><span style="color: #0000DD; font-weight: bold">1</span>)

<span style="color: #888888"># Check if N-2 projection of N-cube contains the origin</span>
<span style="color: #888888"># (thereby checking for plane-cube intersection)</span>
<span style="color: #008800; font-weight: bold">def</span> <span style="color: #0066BB; font-weight: bold">in_plane</span>(q):
    <span style="color: #888888"># Generate list of all N-2 dimensional sub-facets</span>
    <span style="color: #888888"># of unit N-cube centered at q</span>
    sf <span style="color: #333333">=</span> (C<span style="color: #FF0000; background-color: #FFAAAA">@</span>((q<span style="color: #333333">+</span>O)<span style="color: #333333">.</span>T))[:,vxid]<span style="color: #333333">.</span>transpose(<span style="color: #0000DD; font-weight: bold">1</span>,<span style="color: #0000DD; font-weight: bold">2</span>,<span style="color: #0000DD; font-weight: bold">0</span>)
    <span style="color: #008800; font-weight: bold">for</span> v <span style="color: #000000; font-weight: bold">in</span> sf:
        <span style="color: #008800; font-weight: bold">if</span> subfacet_contains(v): <span style="color: #008800; font-weight: bold">return</span> <span style="color: #008800; font-weight: bold">True</span>
    <span style="color: #008800; font-weight: bold">return</span> <span style="color: #008800; font-weight: bold">False</span>

cache <span style="color: #333333">=</span> {}
<span style="color: #008800; font-weight: bold">def</span> <span style="color: #0066BB; font-weight: bold">checkcrystal2</span>(p):
    k <span style="color: #333333">=</span> <span style="color: #007020">tuple</span>(int32(p))
    <span style="color: #008800; font-weight: bold">if</span> <span style="color: #000000; font-weight: bold">not</span> k <span style="color: #000000; font-weight: bold">in</span> cache:
        h <span style="color: #333333">=</span> angle(A<span style="color: #555555; font-weight: bold">@p</span><span style="color: #FF0000; background-color: #FFAAAA">@</span>[<span style="color: #0000DD; font-weight: bold">1</span>,<span style="color: #0000DD; font-weight: bold">1</span>j])
        s <span style="color: #333333">=</span> in_plane(p) <span style="color: #008800; font-weight: bold">if</span> <span style="color: #0000DD; font-weight: bold">0</span><span style="color: #333333">&lt;=</span>h<span style="color: #333333">&lt;=</span>(pi<span style="color: #333333">/</span>N<span style="color: #333333">*</span><span style="color: #6600EE; font-weight: bold">1.1</span>) <span style="color: #008800; font-weight: bold">else</span> checkcrystal2(S<span style="color: #555555; font-weight: bold">@p</span>)
        cache[k]<span style="color: #333333">=</span>s
    <span style="color: #008800; font-weight: bold">return</span> cache[k]

Q,allQ <span style="color: #333333">=</span> {(<span style="color: #0000DD; font-weight: bold">0</span>,)<span style="color: #333333">*</span>N},<span style="color: #007020">set</span>() 
<span style="color: #008800; font-weight: bold">for</span> i <span style="color: #000000; font-weight: bold">in</span> <span style="color: #007020">range</span>(Ndepth):
    Q <span style="color: #333333">=</span> {<span style="color: #007020">tuple</span>(q) <span style="color: #008800; font-weight: bold">for</span> p <span style="color: #000000; font-weight: bold">in</span> Q <span style="color: #008800; font-weight: bold">for</span> q <span style="color: #000000; font-weight: bold">in</span> D<span style="color: #333333">+</span>p <span style="color: #008800; font-weight: bold">if</span> checkcrystal2(q) }
    allQ <span style="color: #333333">|=</span> Q

figure(figsize<span style="color: #333333">=</span>(<span style="color: #0000DD; font-weight: bold">10</span>,<span style="color: #0000DD; font-weight: bold">10</span>))
plotpoints(array(<span style="color: #007020">list</span>(allQ))<span style="color: #333333">.</span>T)
savefig(<span style="background-color: #fff0f0">&quot;qq2_n%d.png&quot;</span><span style="color: #333333">%</span>N)
</pre></td></tr></table></div>


## Create

The above approaches allow you to sample a point set corresponding to the vertex set of a quasicrystaline tiling. You can connect these points to render a rhombic tiling, or calculate mosaic-like Voronoi cells. The tiles can be subdivided, or used as a structure to support detailed elaborations. Get creative, make beautiful things (: